In [759]:
from mmdet3d.visualization import Det3DLocalVisualizer
from mmdet3d.apis import LidarDet3DInferencer

import mmcv
import numpy as np
from mmengine import load

import os.path as osp
import os
from mmdet3d.visualization import Det3DLocalVisualizer
from mmdet3d.structures import CameraInstance3DBoxes, LiDARInstance3DBoxes
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2


In [760]:
def distort_points(points, disto_coeffs):
    # undistort points
    points = points[:, :2]
    
    # Radius for each point
    r = np.sqrt(np.sum(points**2, axis=1))

    def distortion_factors(_r, disto_coeffs):
        disto_factors = np.ones_like(_r)
        for i in range(len(disto_coeffs)):
            disto_factors += disto_coeffs[i] * _r ** (2 * (i + 1))

        return disto_factors
    
    # Compute distortion factors
    disto_factors = distortion_factors(r, disto_coeffs)

    points = points * disto_factors[:, None]

    return points

In [761]:
# cv2_undistored_points = cv2.fisheye.distortPoints(anker_points_camera.T[:, 0:2].reshape(-1, 1, 2), K=camera_matrix, D=distortion_coeffs)

# cv2_undistored_points = cv2_undistored_points.reshape(-1, 2)

# anker_points_camera[0:2, :] = cv2_undistored_points.T

In [762]:
rgb_left = '/home/nicolasstillhard/mmdetection3d_for_rail/data/osdar23_test/orig/8_station_altona_8.2/rgb_left/144_1631700823.500000016.png'
rgb_right = '/home/nicolasstillhard/mmdetection3d_for_rail/data/osdar23_test/orig/8_station_altona_8.2/rgb_right/144_1631700823.500000032.png'
rgb_center = '/home/nicolasstillhard/mmdetection3d_for_rail/data/osdar23_test/orig/8_station_altona_8.2/rgb_center/144_1631700823.500000056.png'
lidar = '/home/nicolasstillhard/mmdetection3d_for_rail/data/osdar23_test/points/8_station_altona_8.2_144.bin'
model = '/home/nicolasstillhard/mmdetection3d_for_rail/checkpoints/rtx4k_pvrcnn_run1/epoch_40.pth'
config = '/home/nicolasstillhard/mmdetection3d_for_rail/checkpoints/rtx4k_pvrcnn_run1/pv_rcnn_8xb2-80e_osdar23-3d.py'

In [ ]:
# Read the image 

img_left = mpimg.imread(rgb_left)
img_right = mpimg.imread(rgb_right)
img_center = mpimg.imread(rgb_center)

# Three subplots sharing both x/y axes

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(img_left)
axs[0].set_title('Left Camera')
axs[1].imshow(img_center)
axs[1].set_title('Center Camera')
axs[2].imshow(img_right)
axs[2].set_title('Right Camera')

plt.show()

In [764]:
gt_bounding_box_path = '/home/nicolasstillhard/mmdetection3d_for_rail/data/osdar23/labels/8_station_altona_8.2_144.txt'


gt_bboxes = pd.read_csv(gt_bounding_box_path, delimiter=' ', header=None)

# Discard the columns 1-7, keep column 0 and 8-14
gt_bboxes = gt_bboxes.iloc[:, [0, 8, 9, 10, 11, 12, 13, 14]]

# rename the bounding boxes


gt_bboxes.columns = ['class', 'dy', 'dz', 'dx', 'x', 'y', 'z', 'yaw']

# only look at 'pedestrian' and 'train' class
gt_bboxes = gt_bboxes[gt_bboxes['class'].isin(['pedestrian', 'train'])]


In [ ]:
coordinate_frame_correction = np.array([[0, -1,0],
                                     [0,0,-1],
                                     [1,0,0]])

rotation_and_translation = np.array([[0.994801848338136, -0.008253767163616355, 0.1014946199098106, 0.05399349045425253],
                                     [0.008530331054488222, 0.9999609911792882, -0.002291194412618652, -0.2858909428211028],
                                      [-0.101471749739171, 0.00314506714465518, 0.9948334496575426, 3.506285581367224]])

rotation_matrix = np.array([[0.994801848338136, -0.008253767163616355, 0.1014946199098106],
                            [0.008530331054488222, 0.9999609911792882, -0.002291194412618652],
                            [-0.101471749739171, 0.00314506714465518, 0.9948334496575426]])

camera_matrix = np.array([[4609.471892628096, 0, 1257.158605934],
                          [0, 4609.471892628096, 820.0498076210201],
                          [ 0, 0, 1]])

# anker_point_coordinates_lidar = gt_bboxes.iloc[:, 4:7].values
# print("Debug: anker_point_coordinates shape: ", anker_point_coordinates_lidar.shape)


anker_point_coordinates_lidar = np.array([[10.05399349045425253, -0.2858909428211028, 3.506285581367224],
                               [10.05399349045425253, -2.2858909428211028, 3.506285581367224],
                               [10.05399349045425253, -4.2858909428211028, 3.506285581367224],
                               [10.05399349045425253, -6.2858909428211028, 3.506285581367224],
                               [10.05399349045425253, 1.7141096, 3.506285581367224],
                               [10.05399349045425253, 3.7141096, 3.506285581367224],
                               [10.05399349045425253, 5.7141096, 3.506285581367224],
                               [10.05399349045425253, -0.2858909428211028, 1.506285581367224],
                               [10.05399349045425253, -0.2858909428211028, -0.493714418632776],
                               [10.05399349045425253, -0.2858909428211028, -2.493714418632776],
                               [10.05399349045425253, -0.2858909428211028, 5.506285581367224],
                               [10.05399349045425253, -0.2858909428211028, 7.506285581367224],
                               [10.05399349045425253, -0.2858909428211028, 9.506285581367224]])

anker_points_lidar_extended = np.hstack((anker_point_coordinates_lidar, np.ones((anker_point_coordinates_lidar.shape[0], 1))))
print("Debug: anker_point_coordinates shape: ", anker_points_lidar_extended.shape)

In [766]:
# camera_matrix_no_centering = np.array([[4609.471892628096, 0, 0],
#                           [0, 4609.471892628096, 0],
#                           [ 0, 0, 1]])

# new_camera_matrix = np.matmul(coordinate_frame_correction, camera_matrix_no_centering)

# # new_camera_matrix[0, 2] = 1257.158605934
# # new_camera_matrix[1, 2] = 820.0498076210201

# new_camera_matrix

In [767]:
# rotation_matrix = np.matmul(coordinate_frame_correction, rotation_matrix)

# rotation_and_translation = np.hstack((rotation_matrix, np.array([[0.05399349045425253], [-0.2858909428211028], [3.506285581367224]])))

In [ ]:
image_height = 1600
image_width = 2464

anker_points_camera = np.matmul(rotation_and_translation, anker_points_lidar_extended.T)

#anker_points_camera = np.matmul(coordinate_frame_correction, anker_points_camera_in_paper)
print("Debug: anker_point_coordinates shape: ", anker_points_camera.shape)

In [769]:
anker_points_image = np.matmul(camera_matrix, anker_points_camera).T

In [ ]:
anker_points_image

In [771]:
anker_points_image = anker_points_image / anker_points_image[:, 2].reshape(-1, 1)

In [772]:
x_coords = anker_points_image[:, 0]
y_coords = anker_points_image[:, 1]

labels = gt_bboxes['class'].values


In [ ]:
img = mpimg.imread(rgb_center)

# Plot the image and the points in a big scatter plot
fig, ax = plt.subplots()

ax.imshow(img)

unique_labels = np.unique(labels)
label_to_numeric = {label: i for i, label in enumerate(unique_labels)}
numeric_labels = np.array([label_to_numeric[label] for label in labels])

# ax.scatter(x_coords, y_coords, marker = 'o', s=10, c=numeric_labels, cmap='winter')
ax.scatter(x_coords, y_coords, marker = 'x', s=50, c='red')

ax.set_xlabel('x')
ax.set_ylabel('y')

scale_factor = 10


# Set x and y limits to the image size
ax.set_xlim(-scale_factor*image_width, image_width*scale_factor)
ax.set_ylim(scale_factor*image_height, -image_height*scale_factor)

# ax.set_xlim(0, scale_factor*image_width*2)
# ax.set_ylim(image_height*scale_factor*2, 0)


# # Set x and y limits to +- 20000
# ax.set_xlim(-5000, 5000)
# ax.set_ylim(5000, -5000)

plt.show()

In [ ]:
cv2_image = cv2.imread(rgb_center)
cv2_image_rgb = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)

for i in range(anker_points_image.shape[0]):
    cv2.circle(cv2_image_rgb, (int(x_coords[i]), int(y_coords[i])), 20, (255, 0, 0), -1)

# Define the axis limits
# plt.xlim(-5000, 5000)
# plt.ylim(5000, -5000)


plt.imshow(cv2_image_rgb)

plt.show()